### Análise de Regras de Associação  de Filmes - Mineração de Dados
___
#### **Objetivo**

Este notebook utiliza mineração de dados para encontrar associações entre filmes bem avaliados (nota ≥ 4.0). O processo envolve pré-processar os dados, criar uma matriz de interações entre usuários e filmes e, em seguida, aplicar o algoritmo Apriori. O objetivo final é gerar e analisar regras de associação para descobrir recomendações de filmes relevantes.
___

#### 1.0 Bibliotecas 

In [1]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori, association_rules

#### 2.0 Carregando os dados

In [2]:
ratings = 'ratings_alterado.csv'
df_ratings = pd.read_csv(ratings,delimiter=';')
movies = 'movies_alterado.csv'
df_movies = pd.read_csv(movies,delimiter=';')
df_combinado = pd.merge(df_ratings, df_movies, on='movieId', how='outer')
     

#### 3.0 Preparação dos Dados

**Considerações iniciais:**

Para que o modelo de regras de associação aprendesse apenas com os filmes que os usuários efetivamente gostaram, foi necessário criar um critério para "preferência".

 - **Critério Adotado:** Foi definido que uma nota igual ou superior a 4.0 (equivalente a 80% da nota máxima) representa uma avaliação positiva.

 - **Justificativa:** Essa nota é um forte indicador de que o usuário gostou do filme, tornando-o uma base confiável para recomendar outros títulos.

 ##### 3.1 Pré-processamento do Dataset

In [3]:
# Filtro apenas boas avaliações
df_combinado = df_combinado[df_combinado['rating'] >= 4.0]
df_combinado.head() 


,userId,movieId,rating,timestamp,title,genres
1,3.0,1,4.0,1.439472e+09,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,5.0,1,4.0,8.586259e+08,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,8.0,1,4.0,8.904925e+08,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
6,12.0,1,4.0,1.167583e+09,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
7,13.0,1,4.0,1.265224e+09,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


 ##### 3.2 Transformação para o Formato Transacional

Para aplicar o algoritmo Apriori, a preparação dos dados é muito importante:

- **Requisito do Algoritmo:** O Apriori opera sobre uma matriz transacional, na qual cada linha deve representar uma "cesta de compras" inteira (ex: todos os filmes que um usuário assistiu).

- **Ação Necessária:** Para atender a essa exigência, a tabela original de interações (formato longo) é pivotada. Esse pivoteamento ocorre utilizando o `userId` como "índice" para criar uma nova linha na tabela de resultados. Utiliza a coluna `title` de filme único para criar uma nova coluna. E utiliza a coluna `ratings` para verificar se o usuário daquela linha avaliou ou não o filme, aqui não é considerado os valores das avaliações. Mas como dito anteriormente, foi feito um filtro das notas as avaliações, que tiveram nota igual ou maior a 4.0.

- **Resultado:** A pivotação transforma os dados na matriz (formato pivoteado) necessária para que o algoritmo possa analisar os conjuntos de itens corretamente. 

- **Conclusão:** Essa transformação para o formato transacional responde a seguinte pergunta:       
"O usuário X avaliou o filme Y?" 
    - `True`: Sim, ele avaliou.
    - `False`: Não, ele não avaliou.

In [4]:
# Criando a tabela pivot para aplicar Apriori no DataFrame
df_filmes_bons = df_combinado.pivot_table(index='userId', columns='title', values='rating', aggfunc=lambda x: True, fill_value=False)
df_filmes_bons = df_filmes_bons.reset_index()
df_filmes_bons.head()

title,userId,"""Great Performances"" Cats (1998)",$ (Dollars) (1971),$5 a Day (2008),'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),...,В движении (2002),Гонгофер (1992),Карусель (1970),Кентервильское привидение (1970),Когда зажигаются ёлки (1950),Начальник,Обезьянки и грабители (1985),Он вам не Димон (2017),Приключения Домовёнка (1986),Принцесса и Людоед (1977)
0,1.0,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,2.0,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,3.0,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,4.0,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,5.0,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


A coluna `userId` é removida do dataset final por duas razões principais:
 - Primeiro, ela já serviu ao seu propósito de organizar os filmes de cada usuário em uma transação (linha). 
 - Segundo, o algoritmo Apriori foca exclusivamente nas relações entre os itens de uma "cesta", a identidade de quem fez a compra (ou assistiu o filme) é irrelavante para a análise, apenas o conteúdo da "cesta de compras".

In [5]:
# Remover a coluna 'userId' para aplicar o Apriori
df_combinado = df_filmes_bons.drop('userId', axis=1,errors='ignore')

#### 4.0 Aplicação do Algoritmo Apriori

 ##### 4.1 Parâmetros de análises

Os parâmetros utilizados para aplicação e análise do algoritmo Apriori foram:

 - **Confiança:** representa a probabilidade de que, entre os usuários que assistiram ao filme X, uma parte percentual também tenha assistido ao filme Y.

 - **Suporte:** indica a frequência mínima com que um conjunto de itens (por exemplo, os filmes X e Y) devem aparecer para ser considerado uma associação relevante.
 
 - **Lift:** mede o grau de correlação entre X e Y, ou seja, a força da associação entre os itens. Quanto maior o lift, mais forte e relevante é a associação para fins de recomendação.
   - `lift > 1` : indica uma associação positiva, ou seja, a presença de um item (filme X) aumenta a probabilidade da ocorrência de outro item (filme Y).
   - `lift = 1:` sugere independência entre os itens (sem valor útil para recomendação).
   - `lift < 1:` sugera uma relação negativa, em que a presença de um item reduz a chance da ocorrência do outro. 

Neste projeto, foram consideradas apenas as regras com lift maior ou igual a 1, focando em identificar combinações de filmes que costumam ser assistidos juntos com frequência significativamente maior do que o esperado ao acaso. Essas associações são especialmente úteis para o desenvolvimento de sistemas de recomendação personalizados.

 ##### 4.2 Definindo o valor min_support para 5%

In [ ]:
# Aplicando Apriori com min_support 5%
frequent_itemsets_05 = apriori(df_combinado, min_support=0.05, use_colnames=True, low_memory=True)
rules_05 = association_rules(frequent_itemsets_05, metric="lift", min_threshold=1.0)

colunas_para_remover = ['representativity', 'leverage', 'conviction', 'zhangs_metric', 'jaccard', 'certainty', 'kulczynski']
rules_5_filtrado = rules_05.drop(columns=colunas_para_remover, axis=1)


 ##### 4.3 Definindo o valor min_support para 10%

In [ ]:
# Aplicando Apriori com min_support 10%
frequent_itemsets_10 = apriori(df_combinado, min_support=0.1, use_colnames=True, low_memory=True)
rules_10 = association_rules(frequent_itemsets_10, metric="lift", min_threshold=1.0)

colunas_para_remover = ['representativity', 'leverage', 'conviction', 'zhangs_metric', 'jaccard', 'certainty', 'kulczynski']
rules_10_filtrado = rules_10.drop(columns=colunas_para_remover, axis=1)


 ##### 4.4 Definindo o valor min_support para 15%

In [ ]:
# Aplicando Apriori com min_support 15%
frequent_itemsets_15 = apriori(df_combinado, min_support=0.15, use_colnames=True, low_memory=True)
rules_15 = association_rules(frequent_itemsets_15, metric="lift", min_threshold=1.0)

colunas_para_remover = ['representativity', 'leverage', 'conviction', 'zhangs_metric', 'jaccard', 'certainty', 'kulczynski']
rules_15_filtrado = rules_15.drop(columns=colunas_para_remover, axis=1)


 ##### 4.5 Definindo o valor min_support para 20%

In [ ]:
# Aplicando Apriori com min_support 20%
frequent_itemsets_20 = apriori(df_combinado, min_support=0.2, use_colnames=True, low_memory=True)
rules_20 = association_rules(frequent_itemsets_20, metric="lift", min_threshold=1.0)

colunas_para_remover = ['representativity', 'leverage', 'conviction', 'zhangs_metric', 'jaccard', 'certainty', 'kulczynski']
rules_20_filtrado = rules_20.drop(columns=colunas_para_remover, axis=1)


#### 5.0 Análise dos Resultados

 ##### 5.1 Comparação dos resultados

In [12]:
# Define uma função para calcular estatísticas básicas de um DataFrame de regras.
def calcular_estatisticas(df):
    """Calcula min, max e média para as colunas 'support', 'confidence' e 'lift'."""
    if df.empty:
        return {'support': [None] * 3, 'confidence': [None] * 3, 'lift': [None] * 3}    
    # Usa "dictionary comprehension" para calcular de forma concisa o min, max e mean
    # para cada uma das colunas de métricas e retorna um dicionário com os resultados.
    return {col: [df[col].min(), df[col].max(), df[col].mean()] for col in ['support', 'confidence', 'lift']}

# Lista dos diferentes valores de suporte mínimo que foram testados.
min_support_values = [0.05, 0.10, 0.15, 0.20]

# Dicionário vazio que será preenchido com os dados para criar o DataFrame de resumo final.
# As chaves do dicionário serão os nomes das colunas.
data = {'Variações do suporte mínimo': [], 'Número de Associações': [], 'Estatísticas': [], 'Suporte': [], 'Confiança': [], 'Lift': []}

# Dicionário que mapeia cada valor de suporte ao seu respectivo DataFrame de regras já filtrado.
rules_dfs = {
    0.05: rules_5_filtrado,
    0.10: rules_10_filtrado,
    0.15: rules_15_filtrado,
    0.20: rules_20_filtrado
}

# Itera sobre cada um dos valores de suporte definidos na lista 'min_support_values'.
for suporte in min_support_values:
    # Pega o DataFrame de regras correspondente ao valor de suporte da iteração atual.
    regras_df = rules_dfs[suporte]
    
    # Chama a função auxiliar para calcular as estatísticas (min, max, média) para este DataFrame.
    stats = calcular_estatisticas(regras_df)
    
    # Calcula o número total de regras (associações) encontradas para este nível de suporte.
    n_assocs = len(regras_df)

    # A lógica a seguir adiciona 3 linhas de dados para cada valor de suporte.
    data['Variações do suporte mínimo'].extend([f'Suporte {int(suporte*100)}%'] + [''] * 2)
    
    # Adiciona o número de associações na primeira linha e duas strings vazias nas seguintes.
    data['Número de Associações'].extend([n_assocs] + [''] * 2)
    
    # Adiciona as labels 'min', 'max', 'média' 
    data['Estatísticas'].extend(['min', 'max', 'média'])
    
    # Para cada métrica (Suporte, Confiança, Lift), adiciona a lista de estatísticas [min, max, média] já formatada.
    # A formatação f'{val:.5f}' garante 5 casas decimais. A verificação 'if val is not None' lida com os casos de DataFrames vazios.
    data['Suporte'].extend([f'{val:.5f}' if val is not None else None for val in stats['support']])
    data['Confiança'].extend([f'{val:.5f}' if val is not None else None for val in stats['confidence']])
    data['Lift'].extend([f'{val:.5f}' if val is not None else None for val in stats['lift']])

# Converte o dicionário 'data', em um DataFrame do pandas.
df_final = pd.DataFrame(data)
df_final

,Variações do suporte mínimo,Número de Associações,Estatísticas,Suporte,Confiança,Lift
0,Suporte 5%,99280,min,0.05002,0.11629,1.02899
1,,,max,0.23320,0.97867,9.35148
2,,,média,0.05914,0.45297,2.91317
3,Suporte 10%,1202,min,0.10004,0.23389,1.21137
4,,,max,0.23320,0.95119,4.68967
5,,,média,0.11916,0.51569,2.13345
6,Suporte 15%,98,min,0.15149,0.35910,1.21347
7,,,max,0.23320,0.93192,4.10476
8,,,média,0.17689,0.59962,2.06119
9,Suporte 20%,12,min,0.20193,0.46944,1.43525


**Conclusões:**
- Observa-se que quanto maior o suporte mínimo, menor o número de associações geradas ( de 99.280 com suporte  5% para apenas 12 com suporte 20%).

- A confiança mínima das associaçoes aumenta significativamente com o aumento do suporte mínimo (de 11,63%  com suporte 5% para 46.94% com suporte 20%).

- O lift mínimo das associações também tende aumentar com o aumento do suporte mínimo     (de 1.02 com suporte 5%  para 1.43 com suporte 20%), o que indica associações mais fortes.

- O suporte máximo se mantém constante (0.23320) em todas as variações, indicando que as regras mais fortes continuam relevantes independentemente do filtro.

 ##### 5.2 Ranking

Foram selecionadas as 5 principais associações, ordenadas por nível de confiança, utilizando o suporte mínimo de `20%` como exemplo. Este valor, o mais alto testado, representa as regras de combinações de itens (filmes) que são mais populares,ou seja, acontecem muitas vezes.

In [ ]:
# Ordenado  por confiança
top_confidence = rules_20_filtrado.sort_values(by='confidence', ascending=False).head()

# Os top 5 para o suporte igual a 20%
print(f"\nTop 5 associações baseadas na Confiança para 20%:")
display(top_confidence)


Top 5 associações baseadas na Confiança para 20%:


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift
10,(Star Wars: Episode V - The Empire Strikes Bac...,(Star Wars: Episode IV - A New Hope (1977)),0.267444,0.310360,0.221827,0.829437,2.672503
11,(Star Wars: Episode IV - A New Hope (1977)),(Star Wars: Episode V - The Empire Strikes Bac...,0.310360,0.267444,0.221827,0.714744,2.672503
6,(Schindler's List (1993)),"(Shawshank Redemption, The (1994))",0.295865,0.430155,0.201933,0.682517,1.586677
8,"(Silence of the Lambs, The (1991))","(Shawshank Redemption, The (1994))",0.352139,0.430155,0.226943,0.644471,1.498231
0,(Forrest Gump (1994)),"(Shawshank Redemption, The (1994))",0.365639,0.430155,0.228933,0.626117,1.455563


#### 6.0 Regras de associação

In [17]:
top_5_confidence_20 = rules_20_filtrado.sort_values(by='confidence', ascending=False).head(5)
print("Top 5 associações baseadas na Confiança para Suporte 20%:\n")
for _, row in top_5_confidence_20.iterrows():
    antecedents = ', '.join(list(row['antecedents']))
    consequents = ', '.join(list(row['consequents']))
    confidence_percentage = row['confidence'] * 100
    print(f"Usuários que gostaram de '{antecedents}' também gostaram de '{consequents}', com {confidence_percentage:.2f}% de confiança.\n")

Top 5 associações baseadas na Confiança para Suporte 20%:

Usuários que gostaram de 'Star Wars: Episode V - The Empire Strikes Back (1980)' também gostaram de 'Star Wars: Episode IV - A New Hope (1977)', com 82.94% de confiança.

Usuários que gostaram de 'Star Wars: Episode IV - A New Hope (1977)' também gostaram de 'Star Wars: Episode V - The Empire Strikes Back (1980)', com 71.47% de confiança.

Usuários que gostaram de 'Schindler's List (1993)' também gostaram de 'Shawshank Redemption, The (1994)', com 68.25% de confiança.

Usuários que gostaram de 'Silence of the Lambs, The (1991)' também gostaram de 'Shawshank Redemption, The (1994)', com 64.45% de confiança.

Usuários que gostaram de 'Forrest Gump (1994)' também gostaram de 'Shawshank Redemption, The (1994)', com 62.61% de confiança.



**Conclusões:**

A apresentação das 5 principais regras de associação, filtradas pelo maior suporte (20%) e ordenadas pela maior confiança, permite extrair as seguintes conclusões:

 - **Validação do Modelo:** 

 A forte associação entre `Star Wars: Episode V` e`Episode IV` demonstra a capacidade do modelo de identificar relações óbvias e esperadas (filmes de uma mesma franquia), servindo como uma validação de sua eficácia.

 - **Aplicação Prática para Recomendações:** 
 
 Essas associações, por possuírem alto suporte e confiança, são candidatas ideais para um sistema de recomendação. Por exemplo, um usuário que avalia positivamente `Forrest Gump`pode receber uma recomendação de alta probabilidade para assistir `The Shawshank Redemption`, com grandes chances de aceitação.

#### 7.0 Insights e Considerações Finais

A análise de regras de associação revelou padrões interessantes nos dados de avaliações, com destaque para o impacto da variação do suporte mínimo nos resultados gerados. A seguir, apresentamos os principais insights:

 **1. Quantidade vs. Qualidade das Regras**
 - Suporte mínimo de 5% gerou o maior número de regras (99.280), porém com menor confiança média (≈45%) e lift médio (≈2.1), indicando muitas associações fracas ou triviais.

 - À medida que o suporte aumenta (10%, 15%, 20%), o número de regras cai drasticamente (até 12 regras em 20%), porém a confiança e o lift médios aumentam, sinalizando associações mais robustas e úteis.

**2. Regras Mais Relevantes**
 - As melhores regras (com confiança > 0.8 e lift > 2.5) foram observadas com suporte mínimo de 20%.

 - Essas regras, apesar de mais escassas, podem indicar fortes padrões de comportamento dos usuários. 

**3. Trade-Off Estratégico**

 - Existe um trade-off claro: usar um suporte mínimo baixo permite identificar padrões mais gerais, porém menos confiáveis; já suportes altos revelam regras específicas e mais confiáveis, ideais para recomendações personalizadas.

**4. Aplicações Possíveis**

  - As regras extraídas podem ser aplicadas para sistemas de recomendação, como:

“Usuários que avaliaram positivamente o Filme A com confiança de 85% também gostaram do Filme B.”

Isso pode ser valioso para plataformas de streaming, sugestões automatizadas, ou marketing direcionado.

___
**Considerações Finais**

Este estudo demonstrou a utilidade do algoritmo Apriori para descobrir padrões de comportamento em avaliações de filmes. A escolha dos parâmetros (como o suporte mínimo) deve ser feita com base no objetivo da análise:

 - Cobertura ampla: suporte baixo (5–10%)

 - Precisão e impacto comercial: suporte médio a alto (15–20%)

 - Em projetos futuros, é possível aprimorar essa análise com:

   - Filtragem por gênero, tempo ou perfil de usuário

   - Integração com sistemas de recomendação baseados em conteúdo

